In [1]:
!pip install fasttext
!wget -O /tmp/lid.176.bin https://dl.fbaipublicfiles.com/fasttext/supervised-models/lid.176.bin

In [2]:
import fasttext
import torch

PRETRAINED_MODEL_PATH = '/tmp/lid.176.bin'
model_lang = fasttext.load_model(PRETRAINED_MODEL_PATH)

In [3]:
import pandas as pd
df=pd.read_csv('../input/pibfulldata/pibfulldata.csv').drop(columns=['Unnamed: 0'])

In [4]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [5]:
from transformers import BartTokenizer, BartForConditionalGeneration
model_name='knkarthick/MEETING_SUMMARY'
tokenizer=BartTokenizer.from_pretrained(model_name)
model=BartForConditionalGeneration.from_pretrained(model_name)
model= model.to(device)

In [6]:
def summary(sent):
    inputs = tokenizer(sent, max_length=150, return_tensors="pt").to(device)
    summary_ids = model.generate(inputs["input_ids"], num_beams=2, min_length=0, max_length=150)
    return tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

In [7]:
import re

def clean_docs(text):
    text=re.sub(r"\t","", text)
    text=re.sub(r" {2,}","", text)
    text=re.sub(r"\xa0","", text)
    index=text.find("*")
    if(index!=-1):
        text=text[:index]
    text=re.sub(r'\n+','\n',text)
    mylist=text.split("\n")
    newlist=[]
    for i in mylist:
        if(len(i)>5):
            i=re.sub(r' .\.[A-Z0-9a-z]\/[A-Z0-9a-z]*', '', i)
            i=re.sub(r'—.*$', '', i)
            pred = model_lang.predict(i, k=2)
            if 'Please Click' in i:
                continue
            if '__label__en' not in pred[0]:
                continue
            newlist.append(i)
    if len(newlist):
        newlist.remove(newlist[0])
    finallist=[]
    for i in newlist:
        if((len(i))>35):
            finallist.append(i)
    return finallist

In [25]:
df_cleaned=pd.DataFrame(columns=['content'])

In [26]:
df_cleaned

# REMOVE 'CLICK HERE' ROWS

In [27]:
count=0

In [28]:
for i in df.content:
    if isinstance(i, str) == False :
        continue
    cleaned = clean_docs(i)
    for i in cleaned:
        count+=1
        if count < 276098:
            continue
        if len(i.split('. ')) > 1:
            i=summary(i)
        df_cleaned.loc[len(df_cleaned)] = i

In [29]:
count

In [30]:
df_cleaned.to_csv('./finalPIBData{}.csv'.format(count))

In [31]:
from IPython.display import FileLink
FileLink('./finalPIBData{}.csv'.format(count))

In [ ]:
df_cleaned.iloc[2].content

In [ ]:
while True:
    pass